In [2]:
from kolmov import crossval_table, crossval_fit_table, get_color_fader, fit_table
from Gaugi import mkdir_p
from sklearn.model_selection import StratifiedKFold, KFold
from multiprocessing import Process
from time import sleep
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import json
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, model_from_json


from pprint import pprint
from copy import deepcopy, copy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00


2022-01-11 11:58:02.647481: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Using all sub packages with ROOT dependence

Applying ATLAS style settings...

Applying ATLAS style settings...


2022-01-11 11:58:03.930790: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-01-11 11:58:03.965292: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-11 11:58:03.965480: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.515GHz coreCount: 14 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 178.84GiB/s
2022-01-11 11:58:03.965503: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-01-11 11:58:03.966707: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-01-11 11:58:03.967369: I tensorflow/stream_executor/platform/d

In [3]:
def my_data_generator( path, pidname='el_lhmedium' ):  
    import pandas as pd
    from kepler import load
    df = load(path)
    # NOTE: Offline filter lhvloose -> lhmedium (as the training procedure)
    df = df.loc[ ((df[pidname]==True) & (df.target==1.0)) | ((df.target==0) & (df['el_lhvloose']==0)) ]
    return df


def my_input_generator( df ):
    '''
    This function will generate the input for ringer models.

    Arguments:
    df: the dataframe
    '''
    col_names= ['trig_L2_cl_ring_%d'%i for i in range(100)]
    rings = df[col_names].values.astype(np.float32)
    
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    rings = norm1(rings)
    return [rings]

def my_dec_generator_after( row, df , remove_last=True):
    
    from Gaugi import load as gload
    file_name  = row.file_name.values[0]
    i_init     = row.init.values[0]
    i_sort     = row.sort.values[0]
    model_idx  = row.model_idx.values[0]
    tuned_data = gload(file_name)['tunedData']
    slope = row.slope.values[0]
    offset = row.offset.values[0]
    for ituned in tuned_data:
        if ((ituned['imodel'] == model_idx)\
            and (ituned['sort'] == i_sort) \
            and (ituned['init'] == i_init)):
            sequence, weights = ituned['sequence'], ituned['weights']
    model = tf.keras.models.model_from_json( json.dumps(sequence, separators=(',', ':')) ) #custom_objects={'RpLayer':RpLayer} )
    model.set_weights( weights )
    if remove_last:
        model = tf.keras.models.Model(model.inputs, model.layers[-2].output)
        
    output = model.predict( my_input_generator(df), batch_size=4096 ).flatten()
    avgmu = df.avgmu.values.flatten()
    avgmu[avgmu>max_avgmu] = max_avgmu
    avgmu[avgmu<min_avgmu] = min_avgmu
    dec = np.greater(output, slope*avgmu + offset)
    return dec

def my_dec_generator( row, df ):
    '''
    This function will load a model from dataframe get their decision and return.

    Arguments:
    row: a dataframe row (used by kolmov)
    df:  the dataframe that will be use to get the decision
    '''
    from Gaugi import load as gload
    file_name  = row.file_name.values[0]
    i_init     = row.init.values[0]
    i_sort     = row.sort.values[0]
    model_idx  = row.model_idx.values[0]
    op_name    = row.op_name.values[0]
    tuned_data = gload(file_name)['tunedData']
    for ituned in tuned_data:
        if ((ituned['imodel'] == model_idx)\
            and (ituned['sort'] == i_sort) \
            and (ituned['init'] == i_init)):
            sequence, weights = ituned['sequence'], ituned['weights']
            thr = ituned['history']['reference']['%s' %(op_name)]['threshold_op']
    model = tf.keras.models.model_from_json( json.dumps(sequence, separators=(',', ':')) ) #custom_objects={'RpLayer':RpLayer} )
    model.set_weights( weights )
    
    output = model.predict( my_input_generator(df), batch_size=4096 ).flatten()
    dec = np.greater(output, thr)
    return dec

In [4]:
etbins  = [4, 7, 10, 15]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]
kf = StratifiedKFold(n_splits=10, random_state=512, shuffle=True)

cv_fit = crossval_fit_table( etbins, etabins, kf, my_data_generator, my_input_generator)

In [5]:
l_path  = '/home/micael/Documents/NeuralRinger/Jpsiee'

tunes_path    = os.path.join(l_path, 'tunes')
analysis_path = os.path.join(l_path, 'analysis')

best_sorts_after = pd.read_csv(os.path.join(analysis_path, 'v1/r0/fitting/best_sorts_v1_with_mu_cor.csv'))
best_sorts_after.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total,slope,offset,fig_signal,fig_background,delta_ref
0,v1.r0.mlp2,0,0,0,6,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.939206,...,21591,27825,25582,28455,215903,-0.010263,-0.108604,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,0.000070
1,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.937434,...,21590,27986,28784,28455,215903,-0.010341,-0.313188,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,0.000035
2,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,tight,0.937434,...,21590,27986,28784,28455,215903,-0.010341,-0.313188,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,0.000035
3,v1.r0.mlp2,0,0,0,8,3,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,vloose,0.941184,...,21590,28148,34616,28455,215903,-0.006295,-0.692980,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,0.000035
4,v1.r0.mlp2,0,1,0,1,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.923447,...,17908,8532,29537,8736,179082,-0.012149,-0.099207,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,0.000114


In [6]:
best_sorts_after.train_tag.unique()

array(['v1.r0.mlp2'], dtype=object)

In [7]:
min_avgmu = 16
max_avgmu = 100

model_name_format = 'data17_13TeV_EGAM2_probes_lhmedium_EGAM7_vetolhvloose.model_v1.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'
op_capnames = ['Tight', 'Medium', 'Loose', 'VeryLoose']
for idx, op in enumerate( ['tight','medium','loose','vloose'] ):
    
    cv_fit.export(best_sorts_after.loc[best_sorts_after.op_name==op], 
                  model_name_format.format(op=op_capnames[idx]), 
                  config_name_format.format(op=op_capnames[idx]), 
                  op, 
                  to_onnx     = True,
                  remove_last = True,
                  min_avgmu   = min_avgmu,
                  max_avgmu   = max_avgmu)

2022-01-11 11:58:34,293 | Py.crossval_fit_table                   INFO Export all tuning configuration to ElectronRingerTightTriggerConfig.conf.
2022-01-11 11:59:00,955 | Py.crossval_fit_table                   INFO Export all tuning configuration to ElectronRingerMediumTriggerConfig.conf.
2022-01-11 11:59:27,618 | Py.crossval_fit_table                   INFO Export all tuning configuration to ElectronRingerLooseTriggerConfig.conf.
2022-01-11 11:59:54,076 | Py.crossval_fit_table                   INFO Export all tuning configuration to ElectronRingerVeryLooseTriggerConfig.conf.


tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The maximum opset needed by this model is only 9.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The maximum opset needed by this model is only 9.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The maximum opset needed by this model is only 9.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The maximum opset needed by this model is only 9.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The maximum opset needed by this model is only 9.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
The ma